In [2]:
import re 
import os
import os.path as osp
import torch
import pandas
import pandas as pd
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import openai
from sklearn.metrics import classification_report

In [3]:
model_name = "beomi/llama-2-koen-13b"

# Fine-tuned model name
new_model = "llama-2-ko-13b-ft-multi-task-t3_l2048_v2"

In [4]:
save_path = 'models/' +  new_model # adapt
save_path 

'models/llama-2-ko-13b-ft-multi-task-t3_l2048_v2'

In [5]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= 'auto',
)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
# 정량 평가용 

# chat 은 0번방이 모델/평가 인풋 , 1번방은 필요없음
# summary 는 0번방이 모델 인풋, 1번방이 평가 인풋, 2번방은 필요없음
# sentiment 는  0번방이 모델 인풋, 1번방이 정답 라벨
chat_single_test, chat_multi2_test, chat_multi3_test = chat_testset_load() # output은 사실 필요없음
summary_test = summary_testset_load() # output은 사실 필요없음, 단 intput은 평가로, instruction은 모델로 해야함. 3개의 값이 필요 
sentiment_test = sentiment_testset_load()  # output 필요함

print('싱글/멀티2-3 데이터셋 =', len(chat_single_test), len(chat_multi2_test) , len(chat_multi3_test) )
print('요약 데이터셋 =', len(summary_test) )
print('분류 데이터셋 =', len(sentiment_test) )
print('총합 데이터셋 =', len(chat_single_test) +  len(chat_multi2_test) + len(chat_multi3_test) + len(summary_test)  +len(sentiment_test) )

싱글/멀티2-3 데이터셋 = 100 100 100
요약 데이터셋 = 200
분류 데이터셋 = 300
총합 데이터셋 = 800


In [10]:
# 싱글/멀티 
single_test = []
multi2_test = []
multi3_test = []

start = time.time()

for x in range(len(chat_single_test)):
    text_input = chat_single_test[x][0]
    result = inference_output(text_input)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    single_test.append(response)

for x in range(len(chat_multi2_test)):
    text_input = chat_multi2_test[x][0]
    result = inference_output(text_input)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    multi2_test.append(response)

for x in range(len(chat_multi3_test)):
    text_input = chat_multi3_test[x][0]
    result = inference_output(text_input)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    multi3_test.append(response)

print("답변 소요시간: {} 초".format(time.time() - start))  # 현재시각 - 시작시간 = 실행 시간

C:\Users\any\anaconda3\envs\oosij\lib\site-packages\transformers\generation\configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


답변 소요시간: 1900.5764889717102 초


In [11]:
# 요약 
s_test = []

start = time.time()

for x in range(len(summary_test)):
    text_input = summary_test[x][0]
    result = inference_output(text_input)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    s_test.append(response)

print("답변 소요시간: {} 초".format(time.time() - start))  # 현재시각 - 시작시간 = 실행 시간

답변 소요시간: 2487.0515520572662 초


In [21]:
s_test[0]

'방탄소년단(BTS)의 ‘다이너마이트’가 콘서트 한 번 없이 온라인 소통만으로 온라인 소통만으로 빌보드 핫100에서 2주간 1위를 차지했다. ‘킹덤’·‘싸이코지만 괜찮아’ 등 한국드라마도 인터넷 동영상 서비스(OTT)를 통해 코로나19 속에서 전 세계의 사랑을 받고 있다. 비대면 시대의 한류는 새로운 기술을 통해 멋진 문화적 만남을 이어주고 있다.'

In [12]:
# 분류 
#X_test = '리스트 타입의 300개의 분류 인풋'
#y_test = '리스트 타입의 평가 데이터셋 300개 정답 아웃풋'
#y_pred = '모델 아웃풋'
y_pred = []
y_test = []

start = time.time()

# 예측 라벨 
for x in range(len(sentiment_test)):
    text_input = sentiment_test[x][0]
    result = inference_output(text_input)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    if response == '긍정':
        response = 2
    if response == '중립':
        response = 1
    if response == '부정':
        response = 0
    y_pred.append(response)

# 정답 라벨
for y in range(len(sentiment_test)):
    y_label = sentiment_test[y][1]
    y_test.append(y_label)

#print(classification_report(y_test, y_pred))

# 예측 결과와 실제 라벨을 매핑하는 딕셔너리
label_mapping = {0: '부정', 1: '중립', 2: '긍정'}

# 실제 라벨을 부정, 중립, 긍정으로 변환
y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_pred

print(classification_report(y_test_mapped, y_pred_mapped))

print("답변 소요시간: {} 초".format(time.time() - start))  # 현재시각 - 시작시간 = 실행 시간

              precision    recall  f1-score   support

           0       0.87      0.94      0.90       100
           1       0.86      0.70      0.77       100
           2       0.81      0.90      0.85       100

    accuracy                           0.85       300
   macro avg       0.85      0.85      0.84       300
weighted avg       0.85      0.85      0.84       300

답변 소요시간: 172.30595350265503 초


In [42]:
## 백업을 위한 저장 

test_dir = './test/'
test_name = 'chat_single'  # summary,  chat_single, chat_multi2, chat_multi3


# 파일로 저장
#with open(test_dir + 'summary_testset_240223.json', 'w', encoding='utf-8') as f:
#    json.dump(s_test, f, ensure_ascii=False, indent=4)


# 불러오기
with open(test_dir + test_name  + '_testset_240223.json', 'r', encoding='utf-8') as f:
    loaded_output_list = json.load(f)

In [11]:
s_test = loaded_output_list

In [ ]:
# GPT 4 기반 테스팅 (G-Eval)

In [30]:
with open('./templates/chat_test.txt', 'r',  encoding='utf-8') as file:
    file_contents = file.read()

In [31]:
print(file_contents)

두 사람 간의 대화가 주어집니다. 다음의 지시문(Instruction)을 받게 될 것입니다. 지시문은 이전 대화내용을 포함하며 현재 대화에 대한 응답(Response)이 제시됩니다.
당신의 작업은 응답을 평가 단계에 따라 응답을 평가하는 것입니다. 
이 평가 기준을 꼼꼼히 읽고 이해하는 것이 중요합니다. 평가하는 동안 이 문서를 계속 열어두고 필요할 때 참조해 주세요. 

평가 기준:
- 이해 가능성 (0 - 1): Instruction에 기반하여 Response를 이해 할 수 있나요?
- 자연스러움 (1 - 5): 사람이 자연스럽게 말할 법한 Instruction 인가요?
- 맥락 유지 (1 - 5): Instruction을 고려했을 때 Response가 맥락을 유지하나요?
- 흥미롭기 (1 - 5): Response가 지루한가요, 아니면 흥미로운가요?
- Instruction 사용 (0 - 1): Instruction에 기반하여 Response를 생성 했나요?
- 공감 능력 (0 - 1): Response에 Instruction의 내용에 기반한 공감의 내용이 있나요?
- 대화 유도 (0 - 1): Response에 질문을 포함하여 사용자의 대답을 자연스럽게 유도하고 있나요?
- 전반적인 품질 (1 - 10): 위의 답변을 바탕으로 이 발언의 전반적인 품질에 대한 인상은 어떤가요?

평가 단계:
1. Instruction, 그리고 Response을 주의깊게 읽습니다.
2. 위의 평가 기준에 따라 Response을 엄격하게 평가합니다.

Instruction:
{{instruction}}

Response:
{{response}}


Result
- 이해 가능성 (0 - 1):
- 자연스러움 (1 - 5):
- 맥락 유지 (1 - 5):
- 흥미롭기 (1 - 5):
- Instruction 사용 (0 - 1):
- 공감 능력 (0 - 1):
- 대화 유도 (0 - 1): 
- 전반적인 품질 (1 - 10):


In [52]:
chat_single_test[0][0]

'이전 대화와 현재 대화의 명령어를 참고하여 가장 적절한 응답을 생성해주세요. 일상적인 대화의 경우, 응답 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.\n\n[이전 대화]\n\n\n[현재 대화]\n### 명령어:\n영화 보는 것을 좋아해요.\n\n### 응답:\n'

In [64]:
loaded_output_list  # 챗봇 명령어 
ins_list = chat_single_test # 챗봇 예상 응답 

#chat_single_score_list = []
chat_multi2_score_list = []
chat_multi3_score_list = []

for f in range(len(loaded_output_list)):
    ints = ins_list[f][0]
    res = loaded_output_list[f]   

    score_result = chat_test_prompt_gpt(ints, res, gpt_model)
    chat_single_score_list.append(score_result)

In [65]:
start = time.time()

test_dir = './test/'
test_name = 'chat_multi2'  # summary,  chat_single, chat_multi2, chat_multi3

# 불러오기
with open(test_dir + test_name  + '_testset_240223.json', 'r', encoding='utf-8') as f:
    loaded_output_list = json.load(f)

loaded_output_list  # 챗봇 명령어 
ins_list = chat_multi2_test # 챗봇 예상 응답 

chat_multi2_score_list = []

for f in range(len(loaded_output_list)):
    ints = ins_list[f][0]
    res = loaded_output_list[f]   

    score_result = chat_test_prompt_gpt(ints, res, gpt_model)
    chat_multi2_score_list.append(score_result)

print("답변 소요시간: {} 초".format(time.time() - start))  # 현재시각 - 시작시간 = 실행 시간

답변 소요시간: 626.0029437541962 초


In [66]:
start = time.time()

test_dir = './test/'
test_name = 'chat_multi3'   # summary,  chat_single, chat_multi2, chat_multi3

# 불러오기
with open(test_dir + test_name  + '_testset_240223.json', 'r', encoding='utf-8') as f:
    loaded_output_list = json.load(f)

loaded_output_list  # 챗봇 명령어 
ins_list = chat_multi3_test # 챗봇 예상 응답 

chat_multi3_score_list = []

for f in range(len(loaded_output_list)):
    ints = ins_list[f][0]
    res = loaded_output_list[f]   

    score_result = chat_test_prompt_gpt(ints, res, gpt_model)
    chat_multi3_score_list.append(score_result)

print("답변 소요시간: {} 초".format(time.time() - start))  # 현재시각 - 시작시간 = 실행 시간

답변 소요시간: 678.5822367668152 초


In [ ]:
### 내일 확인!! ### 

In [81]:
len(chat_single_score_list),len(chat_multi2_score_list), len(chat_multi3_score_list), len(summary_score_list)

(100, 100, 100, 200)

In [34]:
with open('./templates/summary_test.txt', 'r',  encoding='utf-8') as file:
    file_contents = file.read()

In [35]:
file_contents.replace('{{Document}}', doc)

You will be given one summary written for a news article.

Your task is to rate the summary on one metric.

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

[Coherence (1-5)] 
Evaluation Criteria:
- the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby "the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic."

Evaluation Steps:
1. Read the news article carefully and identify the main topic and key points.
2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is

In [13]:
print(summary_test[0][2])

비대면 시대의 한류는 새로운 기술을 통해 멋진 문화적 만남을 이어주고 있다. 미국 타임스지(5. 8)에 따르면 인터넷과 누리관계망(SNS)을 통한 가상연결만으로도 정신건강에 긍정적인 효과가 있다고 한다. 이제는 문화가 대한민국의 브랜드 가치를 높이는 원동력이라는 것을 어느 누구도 부인하지 않는다.


In [12]:
len(summary_test)

200

In [14]:
s_test[0]

'‘킹덤’·‘싸이코지만 괜찮아’ 등 한국드라마도 인터넷 동영상 서비스(OTT)를 통해 코로나19 속에서 전 세계의 사랑을 받고 있다. 비대면 시대의 한류는 새로운 기술을 통해 멋진 문화적 만남을 이어주고 있다.'

In [41]:
# summary_test   0번방 : 명령어,   1번방 : 문서 ,  2번방, 정답 요약
# s_test         예측 요약

summary_score_list = []

for f in range(len(summary_test)): # 200
    doc = summary_test[f][1] # 문서 
    summary = s_test[f] # 예측 요약
    score_result = summary_test_prompt_gpt(doc, summary, gpt_model)
    #file_contents.format(Document = , Summary = )
    summary_score_list.append(score_result)

In [82]:
print(summary_score_list[0])

- Coherence: 3
- Consistency: 5
- Fluency (1-3): 3
- Relevance: 3


In [15]:
print(summary_score_list[0])

- Coherence: 
- Consistency: 
- Fluency (1-3): 
- Relevance:


In [89]:
#output_list = chat_single_score_list  
#output_list = chat_multi2_score_list 
#output_list = chat_multi3_score_list 
output_list = summary_score_list 

In [90]:
## 백업을 위한 저장  -> 전체 결과 저장 

test_dir = './test/'
#test_name = 'chat_single'
#test_name = 'chat_multi2'
#test_name = 'chat_multi3'
test_name = 'summary'


# 파일로 저장
with open(test_dir + test_name + '_result_240223.json', 'w', encoding='utf-8') as f:
    json.dump(output_list, f, ensure_ascii=False, indent=4)
    
print(test_dir + test_name + '_result_240223.json')
    
# 불러오기
#with open(test_dir + 'result_240223.json', 'r', encoding='utf-8') as f:
#    loaded_output_list = json.load(f)

./test/summary_result_240223.json


In [141]:
# - Coherence:
# - Consistency:
# - Fluency (1-3):
# - Relevance:

stop_list =  ['- Coherence:', '- Consistency:', '- Fluency (1-3):', '- Relevance:' ]

total_summary_score_list = []

for s in range(len(summary_score_list)):
    split_score = summary_score_list[s].split('\n')
    score_list = []
    for j in range(len(split_score)):
        clean_score =split_score[j].replace(stop_list[j], '')
        clean_score = float(clean_score.strip())
        score_list.append(clean_score)
    total_summary_score_list.append(score_list)

In [142]:
total_summary_score_list[:10]

[[3.0, 5.0, 3.0, 3.0],
 [2.0, 3.0, 3.0, 2.0],
 [5.0, 5.0, 3.0, 5.0],
 [4.0, 5.0, 3.0, 4.0],
 [4.0, 5.0, 3.0, 4.0],
 [4.0, 5.0, 3.0, 4.0],
 [2.0, 3.0, 3.0, 2.0],
 [5.0, 5.0, 3.0, 5.0],
 [5.0, 5.0, 3.0, 5.0],
 [5.0, 5.0, 3.0, 5.0]]

In [143]:
columns = stop_list

df = pd.DataFrame(total_summary_score_list,  columns=columns) 
df

,- Coherence:,- Consistency:,- Fluency (1-3):,- Relevance:
0,3.0,5.0,3.0,3.0
1,2.0,3.0,3.0,2.0
2,5.0,5.0,3.0,5.0
3,4.0,5.0,3.0,4.0
4,4.0,5.0,3.0,4.0
...,...,...,...,...
195,3.0,5.0,3.0,3.0
196,3.0,5.0,3.0,4.0
197,3.0,5.0,3.0,4.0
198,2.0,5.0,3.0,2.0


In [149]:
# 각 열의 평균 계산
column_means = df.mean().round(1)

print("각 열의 평균:")
# DataFrame을 transpose하여 출력
transposed_df = pd.DataFrame(column_means).transpose()

# 칼럼명 추가
transposed_df.rename(index={0: 'llama2-13b'}, inplace=True)

print(transposed_df)

각 열의 평균:
            - Coherence:  - Consistency:  - Fluency (1-3):  - Relevance:
llama2-13b           3.3             4.4               3.0           3.3


In [147]:
print(column_means.transpose())

- Coherence:        3.3
- Consistency:      4.4
- Fluency (1-3):    3.0
- Relevance:        3.3
dtype: float64


In [107]:
print(chat_single_score_list[0])

- 이해 가능성 (0 - 1): 1
- 자연스러움 (1 - 5): 5
- 맥락 유지 (1 - 5): 5
- 흥미롭기 (1 - 5): 4
- Instruction 사용 (0 - 1): 1
- 공감 능력 (0 - 1): 1
- 대화 유도 (0 - 1): 1
- 전반적인 품질 (1 - 10): 9


In [150]:
## 싱글/멀티 평균 구하기 

# - 이해 가능성 (0 - 1):
# - 자연스러움 (1 - 5):
# - 맥락 유지 (1 - 5):
# - 흥미롭기 (1 - 5):
# - Instruction 사용 (0 - 1):
# - 공감 능력 (0 - 1):
# - 대화 유도 (0 - 1):
# - 전반적인 품질 (1 - 10):

chat_score_list = chat_single_score_list

stop_list =  ['- 이해 가능성 (0 - 1):', '- 자연스러움 (1 - 5):', '- 맥락 유지 (1 - 5):', '- 흥미롭기 (1 - 5):',
             '- Instruction 사용 (0 - 1):', '- 공감 능력 (0 - 1):', '- 대화 유도 (0 - 1):', '- 전반적인 품질 (1 - 10):']

total_chat_score_list = []

for s in range(len(chat_score_list)):
    split_score = chat_score_list[s].split('\n')
    score_list = []
    for j in range(len(split_score[:8])):
        clean_score =split_score[j].replace(stop_list[j], '')
        clean_score = float(clean_score.strip())
        score_list.append(clean_score)
    total_chat_score_list.append(score_list)

In [112]:
stop_list

['- 이해 가능성 (0 - 1):',
 '- 자연스러움 (1 - 5):',
 '- 맥락 유지 (1 - 5):',
 '- 흥미롭기 (1 - 5):',
 '- Instruction 사용 (0 - 1):',
 '- 공감 능력 (0 - 1):',
 '- 대화 유도 (0 - 1):',
 '- 전반적인 품질 (1 - 10):']

In [151]:
len(total_chat_score_list)

100

In [152]:
columns = stop_list

df = pd.DataFrame(total_chat_score_list,  columns=columns) 
df

,- 이해 가능성 (0 - 1):,- 자연스러움 (1 - 5):,- 맥락 유지 (1 - 5):,- 흥미롭기 (1 - 5):,- Instruction 사용 (0 - 1):,- 공감 능력 (0 - 1):,- 대화 유도 (0 - 1):,- 전반적인 품질 (1 - 10):
0,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
1,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
2,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
3,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
4,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
...,...,...,...,...,...,...,...,...
95,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
96,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
97,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
98,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0


In [163]:
# 각 열의 평균 계산
column_means = df.mean().round(1)

# DataFrame을 transpose하여 출력
transposed_df = pd.DataFrame(column_means).transpose()

# 출력
print(transposed_df.to_string(index=False))

 - 이해 가능성 (0 - 1):  - 자연스러움 (1 - 5):  - 맥락 유지 (1 - 5):  - 흥미롭기 (1 - 5):  - Instruction 사용 (0 - 1):  - 공감 능력 (0 - 1):  - 대화 유도 (0 - 1):  - 전반적인 품질 (1 - 10):
               1.0               5.0               5.0              4.0                        1.0               1.0               1.0                  9.0


In [133]:
## 싱글/멀티 평균 구하기 

# - 이해 가능성 (0 - 1):
# - 자연스러움 (1 - 5):
# - 맥락 유지 (1 - 5):
# - 흥미롭기 (1 - 5):
# - Instruction 사용 (0 - 1):
# - 공감 능력 (0 - 1):
# - 대화 유도 (0 - 1):
# - 전반적인 품질 (1 - 10):

chat_score_list = chat_multi2_score_list

stop_list =  ['- 이해 가능성 (0 - 1):', '- 자연스러움 (1 - 5):', '- 맥락 유지 (1 - 5):', '- 흥미롭기 (1 - 5):',
             '- Instruction 사용 (0 - 1):', '- 공감 능력 (0 - 1):', '- 대화 유도 (0 - 1):', '- 전반적인 품질 (1 - 10):']

total_chat_score_list = []

for s in range(len(chat_score_list)):
    split_score = chat_score_list[s].split('\n')
    score_list = []
    for j in range(len(split_score[:8])):
        clean_score =split_score[j].replace(stop_list[j], '')
        clean_score = float(clean_score.strip())
        score_list.append(clean_score)
    total_chat_score_list.append(score_list)

In [134]:
len(total_chat_score_list)

100

In [135]:
columns = stop_list

df = pd.DataFrame(total_chat_score_list,  columns=columns) 
df

,- 이해 가능성 (0 - 1):,- 자연스러움 (1 - 5):,- 맥락 유지 (1 - 5):,- 흥미롭기 (1 - 5):,- Instruction 사용 (0 - 1):,- 공감 능력 (0 - 1):,- 대화 유도 (0 - 1):,- 전반적인 품질 (1 - 10):
0,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
1,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
2,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
3,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
4,1.0,5.0,5.0,4.0,1.0,1.0,0.0,8.5
...,...,...,...,...,...,...,...,...
95,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
96,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
97,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
98,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0


In [136]:
# 각 열의 평균 계산
column_means = df.mean().round(1)

print("각 열의 평균:")
# DataFrame을 transpose하여 출력
transposed_df = pd.DataFrame(column_means).transpose()

# 칼럼명 추가
transposed_df.rename(index={0: 'llama2-13b'}, inplace=True)

print(transposed_df)

각 열의 평균:
- 이해 가능성 (0 - 1):            1.0
- 자연스러움 (1 - 5):             5.0
- 맥락 유지 (1 - 5):             5.0
- 흥미롭기 (1 - 5):              4.0
- Instruction 사용 (0 - 1):    1.0
- 공감 능력 (0 - 1):             1.0
- 대화 유도 (0 - 1):             0.8
- 전반적인 품질 (1 - 10):          8.9
dtype: float64


In [137]:
## 싱글/멀티 평균 구하기 

# - 이해 가능성 (0 - 1):
# - 자연스러움 (1 - 5):
# - 맥락 유지 (1 - 5):
# - 흥미롭기 (1 - 5):
# - Instruction 사용 (0 - 1):
# - 공감 능력 (0 - 1):
# - 대화 유도 (0 - 1):
# - 전반적인 품질 (1 - 10):

chat_score_list = chat_multi3_score_list

stop_list =  ['- 이해 가능성 (0 - 1):', '- 자연스러움 (1 - 5):', '- 맥락 유지 (1 - 5):', '- 흥미롭기 (1 - 5):',
             '- Instruction 사용 (0 - 1):', '- 공감 능력 (0 - 1):', '- 대화 유도 (0 - 1):', '- 전반적인 품질 (1 - 10):']

total_chat_score_list = []

for s in range(len(chat_score_list)):
    split_score = chat_score_list[s].split('\n')
    score_list = []
    for j in range(len(split_score[:8])):
        clean_score =split_score[j].replace(stop_list[j], '')
        clean_score = float(clean_score.strip())
        score_list.append(clean_score)
    total_chat_score_list.append(score_list)

In [138]:
len(total_chat_score_list)

100

In [139]:
columns = stop_list

df = pd.DataFrame(total_chat_score_list,  columns=columns) 
df

,- 이해 가능성 (0 - 1):,- 자연스러움 (1 - 5):,- 맥락 유지 (1 - 5):,- 흥미롭기 (1 - 5):,- Instruction 사용 (0 - 1):,- 공감 능력 (0 - 1):,- 대화 유도 (0 - 1):,- 전반적인 품질 (1 - 10):
0,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
1,1.0,5.0,5.0,4.0,1.0,1.0,0.0,8.5
2,1.0,5.0,5.0,4.0,1.0,1.0,0.0,8.5
3,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
4,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
...,...,...,...,...,...,...,...,...
95,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
96,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0
97,1.0,5.0,5.0,4.0,1.0,1.0,0.0,8.5
98,1.0,5.0,5.0,4.0,1.0,1.0,1.0,9.0


In [140]:
# 각 열의 평균 계산
column_means = df.mean().round(1)

print("각 열의 평균:")
# DataFrame을 transpose하여 출력
transposed_df = pd.DataFrame(column_means).transpose()

# 칼럼명 추가
transposed_df.rename(index={0: 'llama2-13b'}, inplace=True)

print(transposed_df)

각 열의 평균:
- 이해 가능성 (0 - 1):            1.0
- 자연스러움 (1 - 5):             5.0
- 맥락 유지 (1 - 5):             5.0
- 흥미롭기 (1 - 5):              4.0
- Instruction 사용 (0 - 1):    1.0
- 공감 능력 (0 - 1):             1.0
- 대화 유도 (0 - 1):             0.7
- 전반적인 품질 (1 - 10):          8.8
dtype: float64


In [ ]:
## 정성 평가용 

In [8]:
import json

with open('./templates/template_v002.json', 'r',  encoding='utf-8') as file:
    data = json.load(file)

In [20]:
data

{'description': '멀티 턴 대화를 이전 대화 history와 현재 사용자 질문으로 구분해주는 템플릿 입니다. 또한, 2가지 task 분류, 요약을 추가하였습니다.',
 'prompt': '이전 대화와 현재 대화의 명령어를 참고하여 가장 적절한 응답을 생성해주세요. 일상적인 대화의 경우, 응답 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.\n\n[이전 대화]\n{history}\n\n[현재 대화]\n### 명령어:\n{instruction}\n\n### 응답:\n',
 'response_split': '### 응답:'}

In [ ]:
his_log = ''
cnt = 0 
print('무엇이든 질문해보세요! ')
while True:
    query = input('[### 나]: ')
    prompt = data['prompt'].format(history=his_log, instruction=query)
    result = inference_output(prompt)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    print()
    print('[### 심심이봇]: ',response)
    print()
    if cnt == 0:
        his_log = query  +'\n'+ response
    else:
        his_log = his_log +'\n'+query + '\n' + response
    cnt += 1 
    #end = int(input('끝낼꺼면 0 입력'))
    #if end == 0:
    if cnt == 10:
        print('대화종료')
        break

In [61]:
## 불러오기 
os.environ['OPENAI_API_KEY'] = '인증키'

gpt_model = 'gpt-4'


## G-EVAL 평가용 : 싱글/멀티, 요약 

def chat_test_prompt_gpt(instruction, response, gpt_model):
    with open('./templates/chat_test.txt', 'r',  encoding='utf-8') as file:
        file_contents = file.read()

    #p_template = file_contents.format(instruction, response)

    contents = file_contents.replace('{{instruction}}', instruction)
    p_template = contents.replace('{{response}}', response)
    #print(p_template)

    result = get_completion(p_template, gpt_model, temperature=0, verbose=False)
    
    return result

def summary_test_prompt_gpt(Document, Summary, gpt_model):
    with open('./templates/summary_test.txt', 'r',  encoding='utf-8') as file:
        file_contents = file.read()

    contents = file_contents.replace('{{Document}}', Document)
    p_template = contents.replace('{{Summary}}', Summary)

    result = get_completion(p_template, gpt_model, temperature=0, verbose=False)

    return result

# error, retry 추가
def get_completion(prompt, model, temperature=0, verbose=False):
    messages = [{"role": "user", "content": prompt}]
    
    time_start = time.time()
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:    
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=temperature, # this is the degree of randomness of the model's output
                )
                answer = response['choices'][0]['message']['content'].strip()                
                tokens = response.usage.total_tokens                
                
                
                time_end = time.time()
                
                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return answer
            
            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")
                
                if i+1 == retry_count:
                    return prompt, None, None
                time.sleep(4)
                continue


In [6]:
## 최종정리 

# 싱글/멀티 대화턴인 경우
def chat_testset_load():
    test_dir = './data/model_test/'
    test_save_name = 'task_chat_test'
    csv_file_path = test_dir + f"output_{test_save_name}_dataset.csv"
    loaded_data = pd.read_csv(csv_file_path)
    unique_types = loaded_data['type'].unique()
    test_dic = {key: [] for key in unique_types}

    for i in range(len(loaded_data)):
        inst = loaded_data['instruction'][i]
        output = loaded_data['response'][i]
        type = loaded_data['type'][i]
        test_dic[type].append([inst, output])

    result = test_dic['single'], test_dic['multi_2'], test_dic['multi_3']
    return result
    
# 요약인 경우
def summary_testset_load():
    test_dir = './data/model_test/'
    test_save_name = 'task_summarization_test'
    csv_file_path = test_dir + f"output_{test_save_name}_dataset.csv"
    loaded_data = pd.read_csv(csv_file_path)
    unique_types = loaded_data['type'].unique()
    test_dic = {key: [] for key in unique_types}

    for i in range(len(loaded_data)):
        input_ = loaded_data['input'][i]
        inst = loaded_data['instruction'][i]
        output = loaded_data['response'][i]
        type = loaded_data['type'][i]
        test_dic[type].append([inst, input_, output])
        
    return test_dic['task_summarization']
    
# 분류인 경우
def sentiment_testset_load():
    test_dir = './data/model_test/'
    test_save_name = 'task_classification_test'
    csv_file_path = test_dir + f"output_{test_save_name}_dataset.csv"
    loaded_data = pd.read_csv(csv_file_path)
    unique_types = loaded_data['type'].unique()
    test_dic = {key: [] for key in unique_types}

    for i in range(len(loaded_data)):
        inst = loaded_data['instruction'][i]
        output = loaded_data['response'][i]
        type = loaded_data['type'][i]
        test_dic[type].append([inst, output])

    senti_list = test_dic['task_classification']

    # Mapping dictionary for positive, negative, neutral to numbers
    mapping = {'긍정': 2, '중립': 1, '부정': 0}

    # Create an empty list to store the mapped results
    transformed_lst = []

    # Iterate through the list, map each element, and add to the transformed list
    for element in senti_list:
        # Use the first element of the inner list for mapping
        sentiment = element[1]
        transformed_sentiment = mapping.get(sentiment, sentiment)
        transformed_lst.append([element[0], transformed_sentiment])

    return transformed_lst

def inference_output(prompt):
    # 인퍼런스 커스텀
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda") # return_token_type_ids=False
    # peft 일시, **tokenizer(prompt, return_tensors="pt", add_special_tokens=False) 아래 inputs['input_ids'] 교체 , 위는 삭제 
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens = 256,
        do_sample=True,
        top_p=0.9,
        temperature=0.1,
        early_stopping=True,
        eos_token_id= 46332, 
    )
    output = output[0].to("cpu")
    return tokenizer.decode(output)